In [1]:
import ee
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from tqdm.notebook import tqdm, trange
import tqdm
from pprint import pprint 
import statsmodels.api as sm
import math
import geemap
import os

from IPython.display import Image

import folium
from folium import plugins
import importlib

import geopandas as gpd
from PIL import Image

In [2]:
ee.Authenticate()

Enter verification code: 4/1AX4XfWhteMIPuaEh86ewWy4M2zySaIon2tiCjFTxcAhg3r5zYMKp2KZNZMw

Successfully saved authorization token.


In [3]:
ee.Initialize()

This document is meant to create both maps and datasets depicting things like greenspace (NDVI), land surface temperature, and potentially albedo (land reflectance) throughout the city of Chicago or Cook County, organized by zip code (or potentially census tract). If anyone has any questions about the document, please let me know. Otherwise, it should run for everyone.

The below code is used for loading/manipulating/visualizing the NDVI values throughout Chicago. Many thanks to Shopnavo Biswas for both sharing this code and for introducing us all to the entire concept of EarthEngine.

In [4]:
# This code loads the EE dataset which is used to calculate NDVI scores and restricts it to the year 2021

col_sent = ee.ImageCollection("COPERNICUS/S2_SR").filterDate("2021-01-01", "2022-01-01")

In [5]:
# This code loads the EE dataset used to map Chicago zip codes or Cook County and filters the zip codes 
# to include only those in the city of Chicago (with one or two minor exceptions, and not including O'Hare Airport)

# Isolate Cook County from the Census collection
counties = ee.FeatureCollection('TIGER/2018/Counties')
cookCounty = counties.filter(ee.Filter.eq("GEOID", '17031'))

# This is a list of the 67 zip codes that are accepted to make up Chicago
# strict_zip_codes = list(['60007','60018','60068','60106','60131','60176','60601','60602','60603','60604','60605','60606',
#                '60607','60608','60609','60610','60611','60612','60613','60614','60615','60616','60617','60618',
#                '60619','60620','60621','60622','60623','60624','60625','60626','60628','60629','60630','60631',
#                '60632','60633','60634','60636','60637','60638','60639','60640','60641','60642','60643','60644',
#                '60645','60646','60647','60649','60651','60652','60653','60654','60655','60656','60657','60659',
#                '60660','60661','60706','60707','60714','60804','60827'])

# This is a list of the zip codes that show up on a map of Chicago
# Note: Up to 60601 in the above list includes zip codes west of and around O'Hare, 
# which is 60666 in most maps but does not have its own zip code in TIGER/2018/Counties
# and was therefore excluded from this list in order to make as close to a visualization of strictly Chicago as possible
stricter_zip_codes = list(['60601','60602','60603','60604','60605','60606',
               '60607','60608','60609','60610','60611','60612','60613','60614','60615','60616','60617','60618',
               '60619','60620','60621','60622','60623','60624','60625','60626','60628','60629','60630','60631',
               '60632','60633','60634','60636','60637','60638','60639','60640','60641','60642','60643','60644',
               '60645','60646','60647','60649','60651','60652','60653','60654','60655','60656','60657','60659',
               '60660','60661','60707','60827','60666'])

# Get all US zip codes, then restrict to those that intersect with Cook County, and then
# filter out those that don't overlap at all but only touch on the boundary
# (e.g. some adjacent zip codes in Illinois and Indiana)
# Commenting this to form a more strict set of zip codes
# zipCodes = (
#     ee.FeatureCollection('TIGER/2010/ZCTA5').filterBounds(cookCounty)
#     .filter("GEOID10 != '46311'").filter("GEOID10 != '46324'")
#     .filter("GEOID10 != '46321'").filter("GEOID10 != '46327'")
#     .filter("GEOID10 != '46320'").filter("GEOID10 != '60047'")
#     .filter("GEOID10 != '60035'").filter("GEOID10 != '60110'")
#     .filter("GEOID10 != '60118'").filter("GEOID10 != '60143'")
#     .filter("GEOID10 != '60191'").filter("GEOID10 != '60106'")
#     .filter("GEOID10 != '60523'").filter("GEOID10 != '60441'")
#     .filter("GEOID10 != '60448'").filter("GEOID10 != '60449'")
#     .filter("GEOID10 != '60491'").filter("GEOID10 != '60417'")
# )

zipCodes = (
     ee.FeatureCollection('TIGER/2010/ZCTA5').filterBounds(cookCounty)
     .filter(ee.Filter.inList("GEOID10",stricter_zip_codes)))

# Create a geometry that is the union of all the zip codes in Chicago
# I am not sure if this code is necessary
zipChiGeom = zipCodes.union();

#censusTracts = ee.FeatureCollection("TIGER/2010/Tracts_DP1").filterBounds(cookCounty)

# Compiles an array of the GEOID10 (2010 zip codes) column from the zipCodes dataset
zipCodeIDs = zipCodes.aggregate_array("GEOID10").getInfo()

# To view the final list of zip codes
pprint(zipCodeIDs, width=90, compact=True)

['60636', '60601', '60602', '60603', '60604', '60606', '60607', '60612', '60618', '60619',
 '60621', '60631', '60638', '60639', '60641', '60645', '60646', '60651', '60652', '60654',
 '60655', '60656', '60661', '60707', '60647', '60659', '60609', '60634', '60629', '60624',
 '60626', '60827', '60643', '60622', '60637', '60610', '60657', '60615', '60614', '60640',
 '60632', '60653', '60617', '60613', '60611', '60628', '60630', '60608', '60660', '60642',
 '60605', '60633', '60616', '60625', '60620', '60649', '60623', '60644']


In [7]:
# Subset images with less than 5% cloud cover, then subset those which intersect Cook County.
# This isn't strictly necessary, since we will just filter out everything which does not intersect
# Chicago zip codes later on, but it reduces the processing workload of the computer w/o affecting the final result
# I'm actually not even sure if this filterBounds function works, but do not have time to check
clouds = col_sent.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5)).filterBounds(cookCounty)

# Adjust chosen bands to anything you want, and the featurization will flow from it
# We choose the red, noninfrared, green, and blue bands. Learn more at 
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR#bands
chosen_bands = ['B2', 'B3', 'B4', 'B8']

# Select only desired bands from the image 
bands = clouds.select(chosen_bands)
# bands contains all sentinel images which satisfied the above two filters

# Sort the freshly stripped images by CLOUD_COVER variable.
# Note: This is not the same functionas the pandas sort function, and in general earth engine uses all unique fns.
s = bands.sort('CLOUD_COVER')

# Apply earth engine's reduce function (really, a sequence of 4 fns.: reduce(ee.Reducer.statistic()))
# To apply a statistical calculation to all of the columns of data stored in the earth engine object
scene = s.reduce(ee.Reducer.median())
# Get metadata on all images in "s" imageCollection
# s.getInfo()

In [36]:
# Apply the same filters as above, but only to the Near Infared Band (NIR). This code then takes the median of 
# this band over the course of the year, so that the final NDVI value will be a median. This code could be replaced 
# by the simple .median() function, or by the .[statistic] function for a number of statistics, which is just a 
# simplified version of the .reduce(ee.Reducer.[statistic]) function. We had to do this calculation now, 
# rather than after producing the NDVI object, in order to perform basic arithmetic operations on the 
# imageCollections restricted to these bands, because unlike arrays or vectors in pandas, basic arithmetic 
# operations do not work on imageCollections
nir = clouds.select(['B8']).reduce(ee.Reducer.median())

# Do the same for the red band
red = clouds.select(['B4']).reduce(ee.Reducer.median())

# Calculate NDVI (Normalized Difference Vegetation Index) and restrict it to the polygon that is made from
# the union of all Chicago zip codes
numer = nir.add(red)
denom = nir.subtract(red)
ndvi = denom.divide(numer)

# Select only the NDVI values from areas which intersect the areas covered by the Chicago zip codes
# determined above
ndvi_Chi = ndvi.clip(zipCodes.geometry())

# Could have also successfully run
# ndvi_Chi = ndvi.clip(zipChiGeom.geometry()), but again I'm not sure if the zipChiGeom variable is necessary

The below code is for loading and manipulating Land Surface Temperature data.

In [9]:
# A lot of the code in this section is copied from 
# https://developers.google.com/earth-engine/tutorials/community/ph-ug-temp

# Load dataset and select the year 2021 and the boundary CookCounty (the second part did not work)
surfTemp = ee.ImageCollection('MODIS/006/MOD11A2').filterDate('2021-01-01', '2022-01-01').filterBounds(zipCodes)
landSurfaceTemperature = surfTemp.select('LST_Day_1km');
landSurfaceTemperatureVis = {
  'min': 13000.0,
  'max': 16500.0,
  'palette': [
    '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
    '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
    '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
    'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
    'ff0000', 'de0101', 'c21301', 'a71001', '911003'
  ]}

In [31]:
# This is for gettting just one (hopefully representative) image from an entire year's worth of 
# earthengine images. 
# Please feel free to adjust the statistic used to produce that aggregate image.
# Note that there are two datasets for land surface temp.; one is an 8-day mean of daily values and one comprises
# the daily vaules themselves. Access these and other datasets of interest at 
# https://developers.google.com/earth-engine/datasets/catalog/modis

LST_median = landSurfaceTemperature.median()

# Select only the median LST values from areas which intersect the areas covered by the Chicago zip codes
# determined above. This code can be repeated for any of the aggregated images, or for the nonaggregated 
# imageCollection.
LST_median = LST_median.clip(zipCodes.geometry())

LST_mean = landSurfaceTemperature.mean()

LST_max = landSurfaceTemperature.max()

LST_min = landSurfaceTemperature.min()

In [25]:
# For a greenspace map:

# Overview of code:
# Generate a map, add the ndvi as a layer, then center the map on Cook County, and add Chicago's zip codes
# as a layer

# Huge thanks to Matheu for figuring out how to do this

# Define the visual parameters (visParams): basically, just a map from numbers (in this case, NDVI Chicago scores) 
# to colors (the three parameters basically define a color for the lowest possible value and one for the final 
# value — I'm not sure what the third one does; you can play around with these assignments using hexadecimal colors, 
# which can be referenced at https://encycolorpedia.com/000fff).
# These visParams are blue, white, green.
ndviVP = {'min': -1, 'max': 1, 'palette': ['000FFF', 'FFFFFF', '00FF00']}

# Load the map.
fmap = geemap.Map()

# Add the layer with greenspace restricted to Chicago, called ndviChi, with visParams as defined above and entitled
# "Scene 1"
fmap.addLayer(ndvi_Chi, ndviVP, 'Scene 1')

# Center the map output on Cook County
fmap.centerObject(cookCounty, 9)

# This code adds a layer drawing and shading Cook County, which is neither necessary for helpful for our purposes
# at this point. Note that Cook county is a municipal boundary restricted to the state of Illinois, but actually
# overlaps some zip codes in Indiana, and furthermore is much larger than the city of Chicago itself.
# fmap.addLayer(cookCounty)

# This code draws the zip code boundaries on the map. The first input of the draw function determines the color of 
# the lines, the second determines (?), and the third determines the width of the lines. Learn more at
# https://developers.google.com/earth-engine/guides/feature_collections_visualizing.
fmap.addLayer(zipCodes.draw('000000', 0, 1))

# This code produces the map in the code window.
fmap

Map(center=[41.89520774043857, -87.64615767216159], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# For a map of LST or any other dataset loaded from MODIS:

# Load the map
fmap = geemap.Map()

# This centers the map on Cook County and determines the zoom level of the initial image
fmap.centerObject(cookCounty, 9)

# This code simply maps the first measurement in the sequence of 8-day average measurements in the 
# landSurfaceTemperature dataset. In our case, that day was in January, so the surface temperatures 
# were obviously quite cold. The first argument to the addLayer function is an image or an imageCollection, 
# the next is visParams, and the third is the name of the layer, something that you decide.
# fmap.addLayer(
#     landSurfaceTemperature, landSurfaceTemperatureVis,
#     'Land Surface Temperature')

# This code, which is more reasonable, adds a layer with the median LST measurement for every pixel from all of
# the 8-day measurements in the year-long dataset (could have chosen 10-year, or anytime back to when Google 
# started recording data).
# Note:: Should change the visParams to be more interesting/visible, or figure out how to remove the shading caused 
# by adding the zip codes.
fmap.addLayer(
    LST_median, landSurfaceTemperatureVis,
    'Land Surface Temperature')

# I'm not sure what this code does but it is probably from the Uganda or Lyon examples in the EarthEngine tutorials
# lst_urban_point = ee.lst.mean().sample(cookCounty, scale).first().get('LST_Day_1km').getInfo()
# Adds the zip codes — more details above
fmap.addLayer(zipCodes.draw('000000', 0, 1))

# fmap.addLayer()
fmap

In [38]:
# This code will download a csv file containing the data that you select in the geemap.zonal() function
# to your Downloads folder, or to whichever folder you specify in the out_dir function.

# Another huge thank you to Matheu for figuring out how to do this.

# The code in this block has been pre-specified to download a csv of median NDVI values
# By the way, if anyone is curious as to why NDVI equates to greenspace, it has to do with the light emitted by 
# plants, but you should google around to learn more

# Specify final location of the csv file
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

# Specify the name/path of the file itself
# It would be smart to adjust this name based on location, data, and statistic; e.g. "nyc_ndvi_max.csv"
out_stats = os.path.join(out_dir, 'chi_ndvi_medians.csv')

# Create the out_dir location on your computer if it does not already exist
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# This code computes a statistic based on a polygon (in this case, Chicago zip codes) from a given stationary 
# (not time-dependent) dataset (in this case, the ndvi_Chi dataset created above)
# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM
geemap.zonal_statistics(ndvi_Chi, zipCodes, out_stats, statistics_type='MEDIAN', scale=1000)


Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/cweis22/Downloads/chi_ndvi_means.csv


In [27]:
# This is the same code as above, but for downloading median LST values
    
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_stats = os.path.join(out_dir, 'chi_lst_medians.csv')

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

geemap.zonal_statistics(landSurfaceTemperature, zipCodes, out_stats, statistics_type='MEDIAN', scale=1000)


Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /Users/cweis22/Downloads/chi_ndvi_stats.csv
